# **CNN スクラッチ**

In [1]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.
/Users/kobayashishintachi/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kobayashishintachi/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kobayashishintachi/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

In [4]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

In [5]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.fit_transform(y_test[:, np.newaxis])

In [6]:
y_train = y_train_one_hot
y_test = y_test_one_hot

### **【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成**

In [7]:
inp = np.array([2, 3, 5, 7, 4, 2])
fil = np.array([-1, 0, 1])

XS = inp.size
FS = fil.size

OS = XS - FS + 1
x = []

for fs in range(FS+1):
    x.append(np.dot(inp[fs:fs+FS], fil))

x = np.array(x)
print(x+3)

[ 6  7  2 -2]


In [8]:
inp = np.array([2, 3, 5, 7, 4, 2])
fil = np.array([[-1, 0, 1], [0, 3, 1], [2, 1, 0], [5, 8, 1]])
#fil = np.array([-1, 0, 1])

XS = inp.size
FN, FS = fil.shape

OS = XS - FS + 1

xh = np.zeros((FN, OS))
for fs in range(FS+1):
    for fn in range(FN):
        xh[fn][fs] = np.dot(inp[fs:fs+OS-1], fil[fn])
print(xh)

[[ 3.  4. -1. -5.]
 [14. 22. 25. 14.]
 [ 7. 11. 17. 18.]
 [39. 62. 85. 69.]]


In [9]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

In [10]:
da = np.array([10, 20])

In [11]:
db = np.sum(da)
print(db)

dw_0 = np.dot(da, x[0:2])
dw_1 = np.dot(da, x[1:3])
dw_2 = np.dot(da, x[2:4])
# print(dw_0)
# print(dw_1)
# print(dw_2)

dw = np.zeros(w.size)
for i in range(w.size):
    dw[i] = np.dot(da, x[i:i+(x.size-w.size+1)])
print(dw)

dx_0 = da[0] * w[0]
dx_1 = da[0] * w[1] + da[1] * w[0]
dx_2 = da[0] * w[2] + da[1] * w[1]
dx_3 = da[1] * w[2]
print(dx_0)
print(dx_1)
print(dx_2)
print(dx_3)

30
[ 50.  80. 110.]
30
110
170
140


In [12]:
# print(np.dot(x[0:3], w) + b)
# print(np.dot(x[1:4], w) + b)
os = x.size - w.size + 1
xh = np.zeros(os,)
for i in range(os):
    xh[i] = np.dot(x[i:i+w.size], w)
print(xh)

[34. 49.]


In [13]:
class SimpleConv1d:
    def __init__(self, FN, FS, initializer, optimizer):
        self.FN = FN
        self.FS = FS
        self.optimizer = optimizer
        self.initializer = initializer
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.F = initializer.F(self.FN, self.FS)
        self.b = initializer.b(self.FN)
    
    def forward(self, X):
        self.S = X.size
        # 出力サイズ
        self.OS = self.S - self.FS + 1
        if self.FN >= 2:
            # フィルター数、出力サイズの空箱を用意
            xh = np.zeros((self.FN, self.OS))
            for fs in range(self.FS+1):
                for fn in range(self.FN):
                    xh[fn][fs] = np.dot(self.X[fs:fs+OS-1], self.F[fn])
            A = xh + self.b
            
        else:
            xh = np.zeros(self.OS,)
            for i in range(self.OS):
                xh[i] = np.dot(X[i:i+self.FS], self.F)
            A = xh + self.b
        
        return A
    
    def backward(self, dA):
        dF = np.zeros(self.FS)
        for i in range(self.FS):
            dF[i] = np.dot(dA, self.X[i:i+(self.S - self.FS + 1)])
        db = np.sum(dA)
        self = self.optimizer.update(self)
        return dF, db

In [14]:
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
        
    def F(self, FN, FS):
        F = self.sigma * np.random.randn(FN, FS)
        return F
    
    def b(self, FN):
        b = np.random.randn(FN)
        return b

In [15]:
class SGD:
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer):
        layer.F -= self.lr * (layer.dF)
        layer.b -= self.lr * (layer.db)
        return layer.b, layer.b

### **【問題2】1次元畳み込み後の出力サイズの計算**

In [16]:
def out_size(X, F, pad, stride):
    S = X.size
    FS = F.shape[1]
    P = pad
    S = stride
    n_out = ((S + 2*P - FS) / S) + 1
    return n_out

### **【問題3】小さな配列での1次元畳み込み層の実験**

In [17]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

In [18]:
def forward(X, w, b):
    S = X.size
        
    # 出力サイズ
    OS = S - 3 + 1
        
    # フィルター数、出力サイズの空箱を用意
    xh = np.zeros((1, OS))
    for fs in range(4):
        xh[fs] = np.dot(X[fs:fs+3], w)
    A = xh + b
        
    return A

In [19]:
def forward(X, w, b):
        S = X.size
        if w.ndim >= 2:
            OS = S - w.shape[1] + 1
            # フィルター数、出力サイズの空箱を用意
            xh = np.zeros((w.shape[0], OS))
            for fs in range(w.shape[1]+1):
                for fn in range(w.shape[0]):
                    xh[fn][fs] = np.dot(X[fs:fs+OS-1], w[fn])
            A = xh + b
            
        else:
            OS = S - w.size + 1
            xh = np.zeros(OS,)
            for i in range(OS):
                xh[i] = np.dot(X[i:i+w.size], w)
            A = xh + b
        
        return A

In [20]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

In [21]:
forward(x, w, b)

array([35., 50.])

In [22]:
inp = np.array([2, 3, 5, 7, 4, 2])
fil = np.array([[-1, 0, 1], [0, 3, 1], [2, 1, 0], [5, 8, 1]])
bb = np.array([1, 2, 3, 4])

In [23]:
forward(inp, fil, bb)

array([[ 4.,  6.,  2., -1.],
       [15., 24., 28., 18.],
       [ 8., 13., 20., 22.],
       [40., 64., 88., 73.]])

In [24]:
def backward(X, w, b, dA):
        dF = np.zeros(w.size)
        for i in range(w.size):
            dF[i] = np.dot(dA, X[i:i+(X.size - w.size + 1)])
        db = np.sum(dA)
        #self = self.optimizer.update(self)
        return dF, db

In [25]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

delta_a = np.array([10, 20])

In [26]:
backward(x, w, b, delta_a)

(array([ 50.,  80., 110.]), 30)

### **【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成**

In [27]:
np.ones((3, 2, 3))

array([[[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]]])

In [28]:
class SimpleConv1d:
    def __init__(self, FN, FS, initializer, optimizer):
        self.FN = FN
        self.FS = FS
        self.optimizer = optimizer
        self.initializer = initializer
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.F = initializer.F(self.FN, self.FS)
        self.b = initializer.b(self.FN)
    
    def forward(self, X):
        self.S = X.shape[1]
        # 出力サイズ
        self.OS = self.S - self.FS + 1
        if self.FN >= 2:
            # フィルター数、出力サイズの空箱を用意
            xh = np.zeros((self.FN, self.OS))
            for fs in range(self.FS+1):
                for fn in range(self.FN):
                    xh[fn][fs] = np.dot(self.X[fs:fs+OS-1], self.F[fn])
            A = xh + self.b
            
        else:
            xh = np.zeros(self.OS,)
            for i in range(self.OS):
                xh[i] = np.dot(X[i:i+self.FS], self.F)
            A = xh + self.b
        
        return A
    
    def backward(self, dA):
        dF = np.zeros(self.FS)
        for i in range(self.FS):
            dF[i] = np.dot(dA, self.X[i:i+(self.S - self.FS + 1)])
        db = np.sum(dA)
        self = self.optimizer.update(self)
        return dF, db

In [29]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）

In [30]:
xh = np.zeros((3, 2))
y_00 = np.dot(x[0][0:3], w[0][0]) + np.dot(x[1][0:3], w[0][0]) + 1
y_01 = np.dot(x[0][1:4], w[0][0]) + np.dot(x[1][1:4], w[0][1]) + 1
y_10 = np.dot(x[0][0:3], w[1][0]) + np.dot(x[1][0:3], w[1][0]) + 2
y_11 = np.dot(x[0][1:4], w[1][0]) + np.dot(x[1][1:4], w[1][1]) + 2
y_20 = np.dot(x[0][0:3], w[2][0]) + np.dot(x[1][0:3], w[2][0]) + 3
y_21 = np.dot(x[0][1:4], w[2][0]) + np.dot(x[1][1:4], w[2][1]) + 3
print(y_00)
print(y_01)
print(y_10)
print(y_11)
print(y_20)
print(y_21)

16.0
22.0
17.0
23.0
18.0
24.0


In [31]:
def forward(X, w, b):
        S = X.shape[1]
        if w.ndim >= 2:
            OS = S - w.shape[1] + 1
            # フィルター数、出力サイズの空箱を用意
            xh = np.zeros((OS, w.shape[0]))
            for fs in range(OS):
                for fn in range(w.shape[0]):
                    xh[fs][fn] += np.dot(X[fn][fs:fs+OS], w[fs][fn])
            A = xh
            
        else:
            OS = S - w.size + 1
            xh = np.zeros(OS,)
            for i in range(OS):
                xh[i] = np.dot(X[i:i+w.size], w)
            A = xh + b
        
        return A

In [32]:
x = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
print(x[0:2, 0:3])
print(x[0:3, 1:4])

[[1 2 3]
 [5 6 7]]
[[2 3 4]
 [6 7 8]]


In [33]:
x = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])

xh = np.zeros((2, 2, 3))
for oh in range(2):
    for ow in range(2):
        xh[oh, :, :] = x[oh:oh+2, ow:ow+3]
print(xh)

[[[2. 3. 4.]
  [6. 7. 8.]]

 [[6. 7. 8.]
  [6. 7. 8.]]]


In [34]:
def columnzation(x, FH, FW):
    H, W = x.shape
    OH = H - FH + 1
    OW = W - FW + 1
    
    xh = np.zeros((FH, FW, OH, OW))
    for fh in range(FH):
        for fw in range(FW):
            xh[fh, fw, :, :] = x[fh:fh+OH, fw:fw+OW]
    xt = xh.transpose(2, 3, 0, 1)
    return xt.reshape(OH, OW, -1)

In [35]:
x = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
x_col = columnzation(x, 2, 3)
x_co = x_col.reshape(2, 6)
x_co.shape
print(x_col.shape)

(1, 2, 6)


In [36]:
w = np.array([[[1, 2, 3], [4, 5, 6]], [[2, 3, 4], [3, 8, 1]], [[1, 7, 3], [2, 4, 1]]])
w_new = w.reshape(3, -1)
print(w.shape)
print(w_new)

(3, 2, 3)
[[1 2 3 4 5 6]
 [2 3 4 3 8 1]
 [1 7 3 2 4 1]]


In [37]:
print(np.dot(w_new, x_co.T))
print(np.dot(x_co, w_new.T).T)

[[106. 127.]
 [ 90. 111.]
 [ 65.  83.]]
[[106. 127.]
 [ 90. 111.]
 [ 65.  83.]]


In [38]:
x = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
x.shape

(2, 4)

In [39]:
def cforward(x, F, b):
    C, S = x.shape
    FC = F.shape[1]
    FS = F.shape[2]
    OC = C - FC + 1
    OS = S - FS + 1
    
    xh = np.zeros((FC, FS, OC, OS))
    for fh in range(FC):
        for fw in range(FS):
            xh[fh, fw, :, :] = x[fh:fh+OC, fw:fw+OS]
    xt = xh.transpose(2, 3, 0, 1)
    x_col = xt.reshape(OC, OS, -1)
    
    x_col_new = x_col.reshape(C, -1)
    w_new = w.reshape(F.shape[0], -1)
    A = np.dot(x_col_new, w_new.T).T + b.reshape(b.size, -1)
    
    return A

In [40]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])
w = np.ones((3, 2, 3))
b = np.array([1, 2, 3])

In [41]:
cforward(x, w, b)

array([[16., 22.],
       [17., 23.],
       [18., 24.]])

In [42]:
# これはこれでいいけど、1dに特化した形で書いた方がわかりやすい

# class SimpleConv1d:
#     def __init__(self, FN, FS, initializer, optimizer):
#         self.FN = FN
#         self.FS = FS
#         self.optimizer = optimizer
#         self.initializer = initializer
#         # initializerのメソッドを使い、self.Wとself.Bを初期化する
#         self.F = initializer.F(self.FN, self.FS)
#         self.b = initializer.b(self.FN)
    
#     def forward(self, X):
#         C, S = X.shape
#         FC = self.F.shape[1]
#         FS = self.F.shape[2]
#         OC = C - FH + 1
#         OS = S - FW + 1
    
#         xh = np.zeros((FC, FS, OC, OS))
#         for fh in range(FC):
#             for fw in range(FS):
#                 xh[fh, fw, :, :] = X[fh:fh+OC, fw:fw+OS]
#         xt = xh.transpose(2, 3, 0, 1)
#         x_col = xt.reshape(OC, OS, -1)
    
#         x_col_new = x_col.reshape(C, -1)
#         w_new = self.F.reshape(F.shape[0], -1)
#         A = np.dot(x_col_new, w_new.T).T + self.b.reshape(self.b.size, -1)
    
#         return A
    
#     def backward(self, dA):
#         dF = np.zeros(self.FS)
#         for i in range(self.FS):
#             dF[i] = np.dot(dA, self.X[i:i+(self.S - self.FS + 1)])
#         db = np.sum(dA)
#         self = self.optimizer.update(self)
#         return dF, db
    
    
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
        
    def F(self, FN, FS):
        F = self.sigma * np.random.randn(FN, FS)
        return F
    
    def b(self, FN):
        b = np.random.randn(FN)
        return b
    
    
class SGD:
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer):
        layer.F -= self.lr * (layer.dF)
        layer.b -= self.lr * (layer.db)
        return layer.b, layer.b
    
    
class Softmax:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self, x):
        C = np.max(x)
        y = np.exp(x / C) / np.exp(x / C).sum(axis=1).reshape(-1, 1)
        return y
    
    def backward(self, y, t):
        self.y = y
        self.t = t
        batch_size = self.y.shape[0]
        self.loss = -np.sum(t * np.log(y+1e-7)) / batch_size
        dx = (self.y - self.t) / batch_size
        return dx
    
    
class ReLU:
    def __init__(self):
        self.mask = None
        
    def forward(self, X):
        self.mask = X > 0
        A = X * self.mask
        return A
    
    def backward(self, dA):
        dZ = dA * self.mask
        return dZ

In [43]:
class Xavier():
    def __init__(self):
        pass
    
    def W(self, input_size, input_channel, filter_size, n_filters):
        self.sigma = 1 / (np.sqrt(input_size))
        W = self.sigma * np.random.randn(n_filters, input_channel, filter_size)
        
        return W
    
    def b(self, n_filters):
        b = self.sigma * np.random.randn(n_filters)
        
        return b

In [44]:
class SimlpeConv1d_():
    def __init__(self, input_num, filter_num, optimizer, initializer):
        self.padding = 0
        self.stride = 1
        
        self.input_num = input_num
        self.C = self.input_num[0]
        self.N = self.input_num[1]
        
        self.filter_num = filter_num
        self.FN = filter_num[0]
        self.FS = self.filter_num[2]
        
        self.outputs = self.output_size()
        self.initializer = initializer()
        
        self.W = np.ones((3, 2, 3))
        self.b = np.array([1, 2, 3])
        
        self.optimizer = optimizer
        self.indexes = np.array([np.arange(i, i+self.filter_num[2]) for i in range(self.outputs)]).astype(np.int)
        
        self.H_W = 0
        self.H_B = 0
        
    def forward(self, X):
        self.X = X
        
        X_col = np.zeros((self.outputs, self.FS*self.C))
        for i in range(self.C):
            X_col[:self.outputs, i*self.FS:i*self.FS+self.FS] = X[i][self.indexes]
        w_1 = self.W.reshape(-1, self.C*self.FS)
        
        A = (np.dot(X_col, w_1.T) + self.b).T
        return A
    
    def backward(self, dA):
        self.db = dA.sum(axis=1)
        
        indexes = np.array([np.arange(i, i+self.filter_num[2]) for i in range(self.outputs)]).astype(np.int)
        
        dX = np.zeros((2, 4))
        for i in range(self.C):
            w_1 = w.transpose(1, 0, 2)
            X_col = np.dot(w_1[i].T, dA)
            for j in range(self.outputs):
                dX[i, j:j+self.FS] += X_col.T[j,:]
                
        X_col = np.zeros((self.C, self.FS*self.outputs))
        for i in range(self.C):
            X_col[:self.outputs, i*self.FS:i*self.FS+self.FS] = self.X[i][self.indexes]
            
        K = np.dot(dA, X_col)
        self.dW = K.reshape(self.FN, -1, self.FS)
        
        return dX
    
    def output_size(self):
        return (self.input_num[1]+2*self.padding-self.filter_num[2])//self.stride+1

In [45]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])
w = np.ones((3, 2, 3))
b = np.array([1, 2, 3])

In [46]:
class AdaGrad():
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, layer):
        H_W = (layer.H_W + layer.dW * layer.dW)
        H_B = (layer.H_B + layer.db * layer.db)
        
        layer.W -= (self.lr / np.sqrt(H_W)) * layer.dW
        layer.b -= (self.lr / np.sqrt(H_B)) * layer.db
        layer.H_W = H_W
        layer.H_B = H_B

In [47]:
conv1 = SimlpeConv1d_(input_num=(2, 4), filter_num=(3, 2, 3), optimizer=AdaGrad(), initializer=Xavier)

In [48]:
conv1.forward(x)

array([[16., 22.],
       [17., 23.],
       [18., 24.]])

### **【問題5】（アドバンス課題）パディングの実装**

In [49]:
a = np.array([1, 2, 3, 4])
np.pad(a, [1, 1])

array([0, 1, 2, 3, 4, 0])

In [50]:
a = np.arange(1, 5)
b = np.arange(5, 9)
a_pad = np.pad(a, [1, 1])
b_pad = np.pad(b, [1, 1])
c = np.vstack([a_pad, b_pad])
print(c)

[[0 1 2 3 4 0]
 [0 5 6 7 8 0]]


In [51]:
def cforward(x, F, b, stride=1, pad=0):
    C, S = x.shape
    #print(C, S)
    FH = F.shape[1]
    FW = F.shape[2]
    OH = (C + 2*pad - FH) // stride + 1
    OW = (S + 2*pad - FW) // stride + 1
    print('FH:', FH)
    print('FW:', FW)
    print('OH:', OH)
    print('OW:', OW)
    
    if pad != 0:
        x_0 = np.pad(x[0], [pad, pad])
        x_1 = np.pad(x[1], [pad, pad])
        x = np.vstack([x_0, x_1])
        print(x)
    xh = np.zeros((FH, FW, OH, OW))
    print(xh)
    for fh in range(FH):
        for fw in range(FW):
            xh[fh, fw, :, :] = x[fh:fh+OH, fw:fw+OW]
    print('xh:', xh.shape)
    xt = xh.transpose(2, 3, 0, 1)
    x_col = xt.reshape(OH, OW, -1)
    
    x_col_new = x_col.reshape(C, -1)
    w_new = w.reshape(F.shape[0], -1)
    print('x_col_new:', x_col_new.shape)
    A = np.dot(x_col_new, w_new.T).T + b.reshape(b.size, -1)
    
    return A

In [52]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])
w = np.ones((3, 2, 3))
b = np.array([1, 2, 3])

In [53]:
# cforward(x, w, b, stride=1, pad=1)

### 【問題8】学習と推定

In [54]:
class SGD():
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.b -= self.lr * layer.db

In [55]:
class FC():
    
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.initializer = initializer
        self.optimizer = optimizer
        
        self.W = self.initializer.W(n_nodes1, n_nodes2)
        self.b = self.initializer.b(n_nodes2)
        
        self.H_W = 0
        self.H_B = 0
        
    def forward(self, X):
        A = np.dot(X, self.W) + self.b
        self.input = X
        
        return A
    
    def backward(self, dA):
        dZ = np.dot(dA, self.W.T)
        self.dW = np.dot(self.input.T, dA)
        self.db = dA.sum(axis=0)
        
        self = self.optimizer.update(self)
        return dZ

In [56]:
class ReLU():
    def __init__(self):
        self.mask = None
        
    def forward(self, X):
        self.mask = X > 0
        A = X * self.mask
        return A
    
    def backward(self, dA):
        dZ = dA * self.mask
        return dZ

In [57]:
class Softmax():
    def __init__(self):
        pass
    
    def forward(self, X):
        return np.exp(X) / (np.exp(X).sum(axis=1).reshape(-1, 1))
    
    def backward(self, Z, y):
        dA = Z - y
        return dA

In [58]:
# class Xavier():
#     def __init__(self):
#         pass
    
#     def W(self, input_size, input_channel, filter_size, n_filters):
#         self.sigma = 1 / (np.sqrt(input_size))
#         W = self.sigma * np.random.randn(n_filters, input_channel, filter_size)
        
#         return W
    
#     def b(self, n_filters):
#         b = self.sigma * np.random.randn(n_filters)
        
#         return b

In [59]:
class Xavier():
    def __init__(self):
        pass
    
    def W(self, n_nodes1, filter_size):
        self.sigma = 1 / (np.sqrt(n_nodes1))
        W = self.sigma * np.random.randn(filter_size)
        
        return W
    
    def b(self):
        b = self.sigma * np.random.randn()
        
        return b

In [60]:
class He():
    def __init__(self):
        pass
    
    def W(self, n_nodes1, n_nodes2):
        self.sigma = np.sqrt(2 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        
        return W
    
    def b(self, n_nodes2):
        b = self.sigma * np.random.randn(n_nodes2)
        
        return b

In [61]:
class AdaGrad():
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, layer):
        H_W = (layer.H_W + layer.dW * layer.dW)
        H_B = (layer.H_B + layer.db * layer.db)
        
        layer.W -= (self.lr / np.sqrt(H_W)) * layer.dW
        layer.b -= (self.lr / np.sqrt(H_B)) * layer.db
        layer.H_W = H_W
        layer.H_B = H_B

In [62]:
class GetMiniBatch:

    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [63]:
class ScratchDNN:
    def __init__(self, layers, activation, epochs=30, batch_size=100, verbose=True):
        
        self.layers = layers
        self.activation = activation
        
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        
    def fit(self, X, y, X_val=None, y_val=None):
        
        self.loss = []
        self.val_loss = []
        
        # mini_batchを取得
        for i in range(self.epochs):
            
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size)
            for mini_X, mini_y in get_mini_batch:
                
                # 順伝播
                Z = self.forward(mini_X)
                
                #逆伝播
                # softmaxを想定
                dA = self.activation[-1].backward(Z, mini_y)
                dZ = self.layers[-1].backward(dA)
                
                for j in range(2, len(self.layers)+1):
                    dA = self.activation[-j].backward(dZ)
                    dZ = self.layers[-j].backward(dA)
                    
            y_pred = self.forward(X)
            loss_value = self.loss_function(y, y_pred)
            self.loss.append(loss_value)
            
            if X_val is not None:
                y_val_pred = self.forward(X_val)
                val_loss_value = self.loss_function(y_val, y_val_pred)
                self.val_loss.append(val_loss_value)
                
            if self.verbose:
                if X_val is not None:
                    print('epoch{}    loss: {}    val_loss: {}'.format(i+1, loss_value, val_loss_value))
                else:
                    print('epoch{}    loss: {}'.format(i+1, loss_value))
                    
    def predict(self, X):
        pred = self.forward(X).argmax(axis=1)
        
        return pred
    
    def forward(self, X):
        Z = X
        for i in range(len(self.layers)):
            A = self.layers[i].forward(Z)
            Z = self.activation[i].forward(A)
            
        return Z
    
    def loss_function(self, t, y_pred):
        delta = 1e-7
        L = -(t * np.log(y_pred+delta)).sum() / y_pred.shape[0]
        
        return L
    
    def normalize(self, X):
        return (X - X.min()) / (X.max() - X.min())

In [64]:
class SimpleConv1d():
    
    def __init__(self, input_size, filter_size, optimizer, initializer=Xavier):
        
        self.padding = 0
        self.stride = 1
        
        self.input_size = input_size
        self.filter_size = filter_size
        self.output_size = self.output_()
        self.initializer = initializer()
        
        self.W = self.initializer.W(self.input_size, self.filter_size)
        self.b = self.initializer.b()
        
        self.optimizer = optimizer
        
        self.H_W = 0
        self.H_B = 0
        
    def forward(self, X):
        self.X = X
        indexes = np.array([np.arange(i, i+self.filter_size) for i in range(self.output_size)]).astype(np.int)
            
        X = X.reshape(-1)
        return (((X[indexes])*(self.W.reshape(1, -1))).sum(axis=1) + self.b).reshape(1, -1)
        
    def backward(self, dA):
        indexes = np.array([np.arange(i, i+self.filter_size) for i in range(self.output_size)]).astype(np.int)
        self.dW = ((self.X.reshape(-1)[indexes]*(dA.reshape(-1, 1))).sum(axis=0)).reshape(-1)
        self.db = dA.sum()
        dX = np.zeros(self.X.shape[1])
        for i in range(self.output_size):
            dX[i:i+self.filter_size] += (dA[:, i][0])*self.W
        self = self.optimizer.update(self)
            
        return dX
        
    def output_(self):
        return (self.input_size+2*self.padding-self.filter_size)//self.stride+1

In [65]:
class equal():
    
    def forward(self, X):
        return X
    
    def backward(self, dZ):
        return dZ

In [66]:
n_features = 784

sigma = 0.01
layers = []
activation = []
optimizer = AdaGrad(lr=0.01)

layers.append(FC(n_features, 600, He(), optimizer=optimizer))
activation.append(ReLU())
layers.append(FC(600, 400, He(), optimizer=optimizer))
activation.append(ReLU())
layers.append(SimpleConv1d(input_size=400, filter_size=3, optimizer=SGD(), initializer=Xavier))
activation.append(equal())
layers.append(FC(398, 200, He(), optimizer=optimizer))
activation.append(ReLU())
layers.append(FC(200, 10, He(), optimizer=optimizer))
activation.append(Softmax())

In [70]:
dnn = ScratchDNN(layers=layers, activation=activation, epochs=5)
dnn.fit(X_train[0].reshape(1, -1), y_train[0].reshape(1, -1), X_test[0].reshape(1, -1), y_test[0].reshape(1, -1))
y_pred = dnn.predict(X_test)

/Users/kobayashishintachi/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  


epoch1    loss: nan    val_loss: nan
epoch2    loss: nan    val_loss: nan
epoch3    loss: nan    val_loss: nan
epoch4    loss: nan    val_loss: nan
epoch5    loss: nan    val_loss: nan


In [ ]:
X_train.shape